<a href="https://colab.research.google.com/github/MehrdadJalali-AI/BlackHole/blob/main/BH_Server_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from rdkit import RDLogger
from sklearn.tree import DecisionTreeClassifier


# Suppress specific deprecation warnings and RDKit warnings globally
warnings.filterwarnings("ignore", category=DeprecationWarning)
RDLogger.DisableLog('rdApp.*')


def load_edges_and_summary(threshold):
    """
    Loads edge list and summary data based on the given threshold.

    :param threshold: Threshold value for filtering edges.
    :return: Tuple of edges list and summary data.
    """
    edge_file = f"Edges_blackhole_{threshold}.csv"
    summary_file = '1M1L3D_summary.csv'

    # Load edge list (ensure the file exists and has the correct format)
    edges_list = pd.read_csv(edge_file, delimiter=' ', header=None, names=['source', 'target', 'weight'])

    # Load summary data
    summary_data = pd.read_csv(summary_file)

    return edges_list, summary_data
def preprocess_graph(graph, features):
    """
    Processes the graph and aligns the feature matrix with graph nodes.

    :param graph: A NetworkX graph object.
    :param features: Feature matrix as a NumPy array.
    :return: Adjacency matrix and feature matrix.
    """
    feature_dimension = features.shape[1]
    adjacency_matrix = nx.adjacency_matrix(graph).toarray()  # Converts graph to adjacency matrix
    node_to_index = {node: index for index, node in enumerate(graph.nodes())}

    # Align feature vectors with graph nodes
    feature_vectors = [
        features[node_to_index[node]] if node in node_to_index else np.zeros((feature_dimension,))
        for node in graph.nodes()
    ]
    feature_matrix = np.array(feature_vectors)

    return adjacency_matrix, feature_matrix
def save_results_to_csv(accuracies, thresholds, output_file="evaluation_results.csv"):
    """
    Saves evaluation results to a CSV file.

    :param accuracies: Dictionary containing accuracy results for different models.
    :param thresholds: List of threshold values.
    :param output_file: Name of the output CSV file.
    """
    # Convert accuracies dictionary into a DataFrame
    results = pd.DataFrame({'Threshold': thresholds})
    for model_name, model_accuracies in accuracies.items():
        results[model_name] = model_accuracies

    # Save the DataFrame to a CSV file
    results.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

# Helper Functions
def generate_fingerprint(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024), dtype=float)
        return np.zeros((1024,), dtype=float)
    except Exception as e:
        print(f"SMILES Parse Error: {e}")
        return np.zeros((1024,), dtype=float)

def label_encode_metal_names(metal_names):
    metal_dict = {metal: idx for idx, metal in enumerate(np.unique(metal_names))}
    return np.array([metal_dict[metal] for metal in metal_names])

# GCN Model
def build_gcn_model(input_shape_feature, input_shape_adjacency, num_classes):
    x_inp_feature = Input(shape=(input_shape_feature,))
    x_inp_adjacency = Input(shape=(input_shape_adjacency,))

    x_feature = Dense(128, activation='relu')(x_inp_feature)
    x_adjacency = Dense(128, activation='relu')(x_inp_adjacency)

    x = concatenate([x_feature, x_adjacency])
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[x_inp_feature, x_inp_adjacency], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0009), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# GAT Model (simplified version)
def build_gat_model(input_shape_feature, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_shape_feature,)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN Model
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_shape, 1)),  # Add channel dimension for Conv1D
        Conv1D(32, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_and_evaluate_models(X_train, X_test, y_train, y_test, adj_train, feat_train, accuracies, threshold):
    num_classes = y_train.shape[1]

    # GCN
    print ('GCN Model : ')
    gcn_model = build_gcn_model(feat_train.shape[1], adj_train.shape[1], num_classes)
    gcn_model.fit([feat_train, adj_train], y_train, epochs=10, batch_size=64, verbose=1)
    gcn_acc = gcn_model.evaluate([feat_test, adj_test], y_test, verbose=0)[1]
    accuracies['GCN'].append(gcn_acc)

    # GAT
    print ('GAT Model : ')
    gat_model = build_gat_model(X_train.shape[1], num_classes)
    gat_model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)
    gat_acc = gat_model.evaluate(X_test, y_test, verbose=0)[1]
    accuracies['GAT'].append(gat_acc)

    # KNN
    print ('KNN Model : ')
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, np.argmax(y_train, axis=1))
    knn_acc = accuracy_score(np.argmax(y_test, axis=1), knn_model.predict(X_test))
    accuracies['KNN'].append(knn_acc)

    # Gradient Boosting
    print ('Decison Tree : ')
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, np.argmax(y_train, axis=1))
    dt_acc = accuracy_score(np.argmax(y_test, axis=1), dt_model.predict(X_test))
    accuracies['Decision Tree'].append(dt_acc)


    # CNN
    print ('CNN : ')
    cnn_model = build_cnn_model(X_train.shape[1], num_classes)
    X_train_cnn = np.expand_dims(X_train, axis=-1)  # Add channel dimension
    X_test_cnn = np.expand_dims(X_test, axis=-1)
    cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=64, verbose=1)
    cnn_acc = cnn_model.evaluate(X_test_cnn, y_test, verbose=1)[1]
    accuracies['CNN'].append(cnn_acc)



    print(f"Accuracies for threshold {threshold}: GCN={gcn_acc}, GAT={gat_acc}, KNN={knn_acc}, "
          f"Decision Tree={dt_acc}, CNN={cnn_acc}")

from tqdm import tqdm

# Main Program
if __name__ == "__main__":
    thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
    accuracies = {'GCN': [], 'GAT': [], 'KNN': [], 'Decision Tree': [], 'CNN': []}

    for threshold in tqdm(thresholds, desc="Processing Thresholds"):
        print(f"\nProcessing threshold: {threshold}")
        # Load edges and summary data
        edges_list, summary_data = load_edges_and_summary(threshold)

        node_labels_source = edges_list['source'].astype(str).unique()
        node_labels_target = edges_list['target'].astype(str).unique()
        node_labels = np.unique(np.concatenate((node_labels_source, node_labels_target)))

        summary_data_filtered = summary_data[summary_data['refcode'].isin(node_labels)]
        if not summary_data_filtered.empty:
            linker_smiles = summary_data_filtered['linker SMILES']
            if not linker_smiles.empty:
                linker_features = np.stack(linker_smiles.dropna().apply(generate_fingerprint).values)
                metal_names = summary_data_filtered['metal']
                metal_features = label_encode_metal_names(metal_names).reshape(-1, 1)
                other_features = summary_data_filtered[['Largest Cavity Diameter', 'Largest Free Sphere']].values.astype('float32')
                features = np.concatenate((linker_features, metal_features, other_features), axis=1)

                summary_data_filtered['PLD_category'] = pd.cut(
                    summary_data_filtered['Pore Limiting Diameter'],
                    bins=[-np.inf, 2.4, 4.4, 5.9, np.inf],
                    labels=['nonporous', 'small pore', 'medium pore', 'large pore']
                )
                labels = pd.get_dummies(summary_data_filtered['PLD_category']).values

                X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=56)
                graph = nx.read_weighted_edgelist(f"Edges_blackhole_{threshold}.csv")

                adjacency_matrix, feature_matrix = preprocess_graph(graph, features)
                adj_train, adj_test, feat_train, feat_test = train_test_split(adjacency_matrix, feature_matrix, test_size=0.2, random_state=56)

        build_and_evaluate_models(X_train, X_test, y_train, y_test, adj_train, feat_train, accuracies, threshold)


    # Save and visualize results
    save_results_to_csv(accuracies, thresholds)
import matplotlib.pyplot as plt
import pandas as pd

def plot_results_from_csv(csv_file):
    """
    Plots the evaluation results from a CSV file.

    :param csv_file: Path to the CSV file containing evaluation results.
    """
    # Load the CSV file
    results = pd.read_csv(csv_file)

    # Plot each model's accuracy against thresholds
    plt.figure(figsize=(10, 6))

    # Iterate through the columns (excluding the 'Threshold' column)
    for column in results.columns[1:]:
        plt.plot(results['Threshold'], results[column], marker='o', label=column)

    # Customize the plot
    plt.xlabel('Threshold', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.title('Model Accuracies Across Different Thresholds', fontsize=16)
    plt.legend(fontsize=12, title="Models")
    plt.grid(True)
    plt.ylim(0, 1)  # Assuming accuracy values are between 0 and 1
    plt.tight_layout()

    # Show the plot
    plt.show()


Processing Thresholds:   0%|          | 0/11 [00:00<?, ?it/s]


Processing threshold: 0


/tmp/ipykernel_5430/4192754466.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_data_filtered['PLD_category'] = pd.cut(


GCN Model : 
Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5207 - loss: 1.2089
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7740 - loss: 0.6055
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8232 - loss: 0.4906
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8680 - loss: 0.3901
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8924 - loss: 0.3232
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9144 - loss: 0.2641
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9265 - loss: 0.2342
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9378 - loss: 0.1960
Epoch 9/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9528 - loss: 0.1591
Epoch 10/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9617 - loss: 0.1357
GAT Model : 
Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 782us/step - accuracy: 0.5128 - loss: 1.1872
Epoch 2/10
157/157 ━━━━━━

Processing Thresholds:   9%|▉         | 1/11 [00:22<03:45, 22.51s/it]

...

Accuracies for threshold 0.9: GCN=0.7937107086181641, GAT=0.7886792421340942, KNN=0.7823899371069183, Decision Tree=0.8389937106918239, CNN=0.8075471520423889

Processing threshold: 0.95
GCN Model : 
Epoch 1/10


/tmp/ipykernel_5430/4192754466.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_data_filtered['PLD_category'] = pd.cut(


47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4771 - loss: 1.2700 
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - accuracy: 0.7431 - loss: 0.7275
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.8251 - loss: 0.5192
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/step - accuracy: 0.8782 - loss: 0.3911
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.9192 - loss: 0.2996
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - accuracy: 0.9447 - loss: 0.2214
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.9597 - loss: 0.1792
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 965us/step - accuracy: 0.9772 - loss: 0.1357
Epoch 9/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.9831 - loss: 0.1086
Epoch 10/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - accuracy: 0.9895 - loss: 0.0851
GAT Model : 
Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.4441 - loss: 1.5340
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step 

Processing Thresholds: 100%|██████████| 11/11 [03:24<00:00, 18.55s/it]

Accuracies for threshold 0.95: GCN=0.7997311949729919, GAT=0.7997311949729919, KNN=0.7513440860215054, Decision Tree=0.8400537634408602, CNN=0.7983871102333069
Results saved to evaluation_results.csv
